Hubert Wang: Econometrics, junior
- Favorite food is pizza, last ate a week ago from costco (pepperoni)
- Watching tv series with roommates (brooklyn 99, community)


Yu Wan: Computer Science + economics, junior
- Korean style fried chicken
- Watching tv series with roommates (breaking bad, chinese tv shows)

## Workshop - Regression-Based Classification

Does `statsmodels` marginal effect use the average of covariates or the average predicted values? 
- Use the class data.
- Show your work.

Load the necessary packages and data:

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm # progress bar

import statsmodels.api as sm
from sklearn import linear_model as lm

from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc = {'axes.titlesize': 24,
             'axes.labelsize': 20,
             'xtick.labelsize': 12,
             'ytick.labelsize': 12,
             'figure.figsize': (8, 4.5)})
sns.set_style("white") # for plot_confusion_matrix()

In [3]:
df = pd.read_pickle('C:/Users/hubst/Econ490_group/class_data.pkl')

df_prepped = df.drop(columns = ['urate_bin', 'year', 'GeoName']).join([
    pd.get_dummies(df['urate_bin'], drop_first = True),
    pd.get_dummies(df.year, drop_first = True)    
])

y = df_prepped['pos_net_jobs'].astype(float)
x = df_prepped.drop(columns = 'pos_net_jobs')

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 2/3, random_state = 490)

x_train_std = x_train.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)
x_test_std  = x_test.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)

x_train_std = sm.add_constant(x_train_std)
x_test_std  = sm.add_constant(x_test_std)
x_train     = sm.add_constant(x_train)
x_test      = sm.add_constant(x_test)

Fit a logistic regression using either `sm.Logit()` or `smf.logit()`.

In [4]:
fit_logit = sm.Logit(y_train, x_train).fit()

Optimization terminated successfully.
         Current function value: 0.598795
         Iterations 6


Get the marginal effects (`.get_margeff()`). Print the summary (`.summary()`).

In [8]:
print(fit_logit.get_margeff())
print(fit_logit.summary())

                           Logit Regression Results                           
Dep. Variable:           pos_net_jobs   No. Observations:                33418
Model:                          Logit   Df Residuals:                    33390
Method:                           MLE   Df Model:                           27
Date:                Tue, 02 Mar 2021   Pseudo R-squ.:                  0.1267
Time:                        14:44:25   Log-Likelihood:                -20011.
converged:                       True   LL-Null:                       -22913.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -2.1336      0.149    -14.285      0.000      -2.426      -1.841
pct_d_rgdp            0.0155      0.001     10.386      0.000       0.013       0.018
emp_estabs            0.

In [17]:
np.mean(x)

pct_d_rgdp                1.950070
emp_estabs               13.359081
estabs_entry_rate         9.299141
estabs_exit_rate          8.880961
pop                  103177.530373
pop_pct_black             9.530799
pop_pct_hisp              8.155588
lfpr                     76.466245
density                 253.024034
lower                     0.379017
similar                   0.183713
2003                      0.059130
2004                      0.058831
2005                      0.059309
2006                      0.059748
2007                      0.059010
2008                      0.059190
2009                      0.058791
2010                      0.058372
2011                      0.059050
2012                      0.058352
2013                      0.058432
2014                      0.058511
2015                      0.058511
2016                      0.058531
2017                      0.058252
2018                      0.058392
dtype: float64

***
# Covariate Averages
$$
\frac{\partial p(x_i)}{\partial \beta_1} \approx \frac{e^{\hat{\beta}_0 + \bar{x}\hat{\beta}_1 + \bar{x}\hat{\beta_2}}}{(1 + e^{\hat{\beta}_0 + \bar{x}\hat{\beta}_1 + \bar{x}\hat{\beta_2}})^2}\hat{\beta}
$$

In [13]:
exp(-2.1336 + 0.0155 + 0.0417 + 0.2041 + (-0.1647) + 0.00000009949 + (-0.0033) 
    + 0.0059 + 0.0019 + 0.000008225 + 0.3610 + 0.1662 + 1.0495 + 1.3234 + 1.2072 
    + 1.7648 + 0.5726 + 0.8626 + (-0.5327) + 0.2174 + 1.3064 + 1.5542 + 1.2692 + 1.5313 + 1.6098 + 1.3157 + 1.1677 + 1.4274)

NameError: name 'average' is not defined

***
# Predicted values Averages
$$
\frac{\partial p(x_i)}{\partial \beta_1} \approx \frac{1}{n} \sum_{i=1}
^n \frac{e^{\hat{y}_i}}{1 + e^{\hat{y}_i}}\hat{\beta}
$$

*** 
# Interpretaton

Interpret the marginal effect on one feature.